<p align= " center"> <strong>SPATIAL AUTOREGRESSIVE MODEL</p>

<p><b>By: Jefferson C.</b></p>

---

### __SAR Model__

__Def.__

Modelo econométrico espacial usado para __modelar la dependencia espacial entre las observaciones de diferentes ubicaciones geográficas.__ 

Se asume que los valores de la variable dependiente en una ubicación están influenciados por los valores de la variable dependiente en las ubicaciones cercanas.

Model

- __General Form__
$$y_i = \rho \sum_{j=1}^{N} w_{ij} y_j + X_i \beta + \epsilon_i$$

__Donde:__
- $y_i$: variable dependiente en $i$,
- $w_{ij}$: peso espacial entre $i$ y $j$,
- $\rho$: parámetro de influencia espacial,
- $X_i$: variables explicativas en $i$,
- $\epsilon_i$: error en $i$.

equivalentemente 
- __Matrix Form__

$$y = \rho W y + X \beta + \epsilon$$

Donde:
- $y$: vector de observaciones de $y_i$,
- $W$: matriz de pesos espaciales,
- $\rho$: parámetro de dependencia espacial,
- $X$: matriz de variables explicativas,
- $\beta$: coeficientes de $X$,
- $\epsilon$: vector de errores.


In [2]:
# IMPORT DATA
import geopandas as gpd

# Summary data
data = gpd.read_file("../Data Bases/healthIndicators")
data.describe()

,shape_area,shape_len,BirthRate,Gener_Rate,LowBi_ight,Prena_ster,Prete_rths,TeenB_Rate,Assau_cide,Breas_ales,...,Chil_ing_1,Gonor_ales,Gono_les_1,Tuber_osis,Below_evel,Crowd_sing,Dependency,NoHig_loma,PerCa_come,Unemp_ment
count,7.700000e+01,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,...,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,8.361453e+07,44397.606964,15.698701,67.974026,9.623377,76.506494,11.264935,50.064935,18.068831,25.951948,...,0.851948,754.918182,709.651948,6.844156,20.292208,4.912987,35.829870,21.596104,25106.740260,13.303896
std,5.494626e+07,20090.463816,3.528735,15.264315,3.953809,5.243099,3.016916,28.097817,16.561077,9.557590,...,0.774021,883.757117,774.444705,4.587956,11.496988,3.657341,7.269802,12.354995,14952.672297,7.031965
min,1.691396e+07,18137.944253,9.400000,27.000000,3.000000,63.000000,5.000000,1.300000,0.000000,7.600000,...,0.000000,0.000000,0.000000,0.000000,3.100000,0.200000,15.500000,2.900000,8535.000000,4.200000
25%,4.976964e+07,31948.598840,12.900000,60.000000,7.000000,73.000000,8.800000,33.700000,4.900000,20.200000,...,0.300000,93.100000,92.200000,3.000000,12.000000,2.000000,32.300000,13.400000,15467.000000,7.800000
50%,7.963575e+07,43229.372704,15.700000,68.000000,8.000000,76.000000,10.800000,49.200000,10.800000,24.000000,...,0.700000,216.600000,318.700000,6.500000,18.200000,4.200000,38.300000,18.500000,20489.000000,11.500000
75%,9.885317e+07,49478.427771,18.500000,80.000000,12.000000,80.000000,13.700000,67.900000,32.200000,32.700000,...,1.100000,1298.200000,1397.900000,9.400000,26.100000,6.800000,40.900000,29.400000,29026.000000,17.400000
max,3.718356e+08,173625.984660,22.400000,94.000000,19.000000,94.000000,17.500000,116.900000,70.300000,54.700000,...,3.700000,3193.300000,2545.700000,22.700000,61.400000,17.600000,50.200000,58.700000,87163.000000,40.000000


In [3]:
# SPATIAL AUTOREGRESSIVE MODEL 

from  pysal.lib import weights
import spreg 
from spreg import ml_lag

# Weight Matrix
W = weights.Queen.from_dataframe(data,use_index=True)
W.transform = 'r'

# Dependent Variable 
Y = data['BirthRate']

# Independent Variable
X =data[['Infan_Rate','Tuber_osis']]

# Model 
sar_model = ml_lag.ML_Lag(Y,X,w=W)
print(sar_model.summary)

# Residuals
data["RESIDUALS"]= sar_model.u

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :   BirthRate                Number of Observations:          77
Mean dependent var  :     15.6987                Number of Variables   :           4
S.D. dependent var  :      3.5287                Degrees of Freedom    :          73
Pseudo R-squared    :      0.2857
Spatial Pseudo R-squared:  0.0989
Log likelihood      :   -195.9229
Sigma-square ML     :      8.9493                Akaike info criterion :     399.846
S.E of regression   :      2.9915                Schwarz criterion     :     409.221

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------------------------------------------------------------

### __Assumptions__

### 1. __Stationarity__

__Las propiedades estadísticas del proceso (media, varianza y covarianza) no cambian a lo largo del espacio.__ Es decir, los patrones espaciales deben ser constantes entre ubicaciones.

__Mathematical Formulation__

La estacionariedad se expresa en términos de la covarianza entre las ubicaciones $i$ y $j$:

$$\text{Cov}(y_i, y_j) = \gamma(d_{ij})$$

donde $\gamma(d_{ij})$ depende solo de la distancia $d_{ij}$ entre las ubicaciones, no de las ubicaciones mismas.

__Hypothesis__

- $H_0)$ El proceso es no estacionario.

  $$H_0: \rho = 1 \quad (\text{raíz unitaria presente, implica no estacionariedad})$$

- $H_1)$ El proceso es estacionario.

  $$H_1: |\rho| < 1 \quad (\text{sin raíz unitaria, implica estacionariedad})$$

__Test__

En la prueba de __Dickey-Fuller__, si $\rho \neq 1$, se rechaza $H_0$ y se concluye que el proceso es estacionario.

In [10]:
# AUGMENTED DICKEY-FULLER (ADF) TEST

from statsmodels.tsa.stattools import adfuller
import numpy as np

# Apply test to a time series
y_ts = np.asarray(Y).flatten()

# Test (autolag chooses lag length automatically)
adf_result = adfuller(y_ts, autolag="AIC")

# Output
print("AUGMENTED DICKEY-FULLER (ADF) TEST \n")
print(f"ADF stat : {adf_result[0]:.4f}")
print(f"p-value  : {adf_result[1]:.4f}\n")

# Decision
alpha = 0.05
if adf_result[1] < alpha:
    print("Reject H0: series is stationary.")
else:
    print("Fail to reject H0: series is non-stationary (unit root).")

AUGMENTED DICKEY-FULLER (ADF) TEST 

ADF stat : -6.0705
p-value  : 0.0000

Reject H0: series is stationary.


In [5]:
# DICKER-FULLER TEST

from statsmodels.tsa.stattools import adfuller

# Apply test to Dependent Variable (Y)
adf_result = adfuller(Y)

# Results 
print(f"Statistical Dickey-Fuller: {adf_result[0]:3f}")
print(f"P-value: {adf_result[1]:.3f} \n\nDecision:")

# Decision
if adf_result[1] < 0.05:
    print("Reject Ho")
else:
    print("Do not reject Ho")

Statistical Dickey-Fuller: -6.070504
P-value: 0.000 

Decision:
Reject Ho


### __2. Independence of errors__

__Los errores en diferentes ubicaciones deben ser no correlacionados.__ Es decir, los errores no deben mostrar ninguna relación espacial entre las ubicaciones.

__Mathematical Formulation__

La independencia de los errores se verifica comprobando que la covarianza entre los errores en diferentes ubicaciones sea cero:

$$\text{Cov}(\epsilon_i, \epsilon_j) = 0 \quad \text{para} \quad i \neq j$$

__Hypothesis__

- $H_0)$ Los errores están correlacionados espacialmente.

  $$H_0: \text{Cov}(\epsilon_i, \epsilon_j) \neq 0 \quad (\text{autocorrelación espacial presente})$$

- $H_1)$ Los errores son independientes.

  $$H_1: \text{Cov}(\epsilon_i, \epsilon_j) = 0 \quad (\text{sin autocorrelación espacial})$$

__Test__

Para probar este supuesto, se utilizan pruebas de autocorrelación espacial, como el __Índice de Moran o pruebas LM__ de autocorrelación espacial, para detectar si existe correlación espacial entre los errores.

In [ ]:
# MORAN'S I (Spatial autocorrelation in residuals)

import numpy as np
import libpysal
from esda.moran import Moran

# Residuals (vector)
resid = np.asarray(data["RESIDUALS"]).flatten()

# Weights for Moran
# If W is already a weights object, use it. If W is a sparse matrix, convert it.
w_moran = W if hasattr(W, "neighbors") else libpysal.weights.WSP(W).to_W()
w_moran.transform = "r"

# Moran's I
moran = Moran(resid, w_moran, permutations=999)

# Results
print("MORAN'S I RESULTS \n")
print(f"I      : {moran.I:.3f}")
print(f"p-value: {moran.p_sim:.3f}\n")

# Decision
alpha = 0.05
if moran.p_sim < alpha:
    print("Reject H0: significant spatial autocorrelation in the errors (dependence).")
else:
    print("Fail to reject H0: no significant spatial autocorrelation in the errors (independence).")

MORAN'S I RESULTS 

I      : -0.002
p-value: 0.428

Fail to reject H0: no significant spatial autocorrelation in the errors (independence).


In [14]:
# LAGRANGE MULTIPLIER (LM-ERROR) TEST

import numpy as np
import libpysal
import spreg

# Weights (must be libpysal.weights.W)
w_diag = W if hasattr(W, "neighbors") else libpysal.weights.WSP(W).to_W()
w_diag.transform = "r"

# LM-Error test (typically run on an OLS model object)
cache = spreg.diagnostics_sp.spDcache(sar_model, w_diag)
lm_stat, lm_p = spreg.diagnostics_sp.lmErr(sar_model, w=w_diag, spDcache=cache)

# Results
print("LAGRANGE MULTIPLIER (LM-ERROR) TEST \n")
print(f"LM stat  : {lm_stat:.4f}")
print(f"p-value  : {lm_p:.4f}\n")

# Decision
alpha = 0.05
if lm_p < alpha:
    print("Reject H0: significant spatial autocorrelation in the errors (dependence).")
else:
    print("Fail to reject H0: no significant spatial autocorrelation in the errors (independence).")

LAGRANGE MULTIPLIER (LM-ERROR) TEST 

LM stat  : 0.0006
p-value  : 0.9806

Fail to reject H0: no significant spatial autocorrelation in the errors (independence).


### __3. Homoscedasticity__

__Implica que la varianza de los errores es constante en todas las ubicaciones espaciales.__ Esto asegura que la dispersión de los errores no cambie según la ubicación, garantizando la consistencia del modelo.

__Mathematical Formulation__

La homocedasticidad se expresa a través de la varianza de los errores, que debe ser constante en todas las ubicaciones $i$:

$$\text{Var}(\epsilon_i) = \sigma^2 \quad \text{para todas las ubicaciones } i$$

Esto implica que la varianza de los errores $\epsilon_i$ es igual a una constante $\sigma^2$ y no depende de la ubicación $i$.

__Hypothesis__

- $H_0)$ Los errores son homocedásticos, la varianza de los errores es constante en todas las ubicaciones.

  $$ H_1: \text{Var}(\epsilon_i) = \sigma^2 \quad (\text{varianza constante, homocedasticidad}) $$



- $H_1)$  Los errores son heterocedásticos, la varianza de los errores varía entre ubicaciones.

  $$ H_0: \text{Var}(\epsilon_i) \neq \sigma^2 \quad (\text{varianza no constante, heterocedasticidad}) $$

__Test__

Para verificar la homocedasticidad, se puede utilizar la prueba de __Breusch-Pagan o la prueba SABP (Spatially Adjusted Breusch-Pagan)__, que detecta la heterocedasticidad espacial en los errores del modelo.

In [16]:
# SPATIALLY ADJUSTED BREUSCH–PAGAN (SABP) TEST

from spreg import breusch_pagan

# Test
sabp_test = breusch_pagan(sar_model)   # works with spreg regression objects (OLS/ML_Lag/ML_Error/etc.)

# Results
print("SPATIALLY ADJUSTED BREUSCH-PAGAN (SABP) TEST \n")
print(f"SABP stat : {sabp_test['bp']:.4f}")
print(f"p-value   : {sabp_test['pvalue']:.4f}")
print(f"df        : {sabp_test['df']}\n")

# Decision
alpha = 0.05
if sabp_test["pvalue"] < alpha:
    print("Reject H0: evidence of heteroscedasticity.")
else:
    print("Fail to reject H0: no evidence of heteroscedasticity.")

SPATIALLY ADJUSTED BREUSCH-PAGAN (SABP) TEST 

SABP stat : 0.6789
p-value   : 0.7122
df        : 2

Fail to reject H0: no evidence of heteroscedasticity.


### __4. No Multicollinearity__

__Las variables explicativas del modelo no deben estar perfectamente correlacionadas entre sí.__ Si dos o más variables están altamente correlacionadas, los coeficientes del modelo se estiman de manera imprecisa, dificultando la interpretación.

__Mathematical Formulation__

La multicolinealidad se evalúa observando las correlaciones entre las variables explicativas $X_i$. Para evitarla, debe cumplirse que:

$$\text{Cor}(X_i, X_j) \neq 1 \quad \text{para} \quad i \neq j$$

Esto significa que la correlación entre las variables explicativas $X_i$ y $X_j$ no debe ser 1, ya que una correlación perfecta implica multicolinealidad perfecta.

__Hypothesis__

- $H_0)$ Existen altas correlaciones entre las variables explicativas, lo que indica multicolinealidad.

  $$ H_0: \text{Cor}(X_i, X_j) \approx 1 \quad (\text{multicolinealidad presente}) $$

- $H_1)$ No existen correlaciones significativas entre las variables explicativas, lo que implica que no hay multicolinealidad.

  $$ H_1: \text{Cor}(X_i, X_j) \neq 1 \quad (\text{sin multicolinealidad}) $$

__Test__

Se utiliza el __Índice de Inflación de la Varianza (VIF)__ para medir la multicolinealidad. Un valor alto de VIF indica que una variable está altamente correlacionada con otras y puede causar problemas en la estimación del modelo.


In [9]:
# VARIANCE INFLATION INDEX
 
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Add constatnt
X_const = sm.add_constant(X)

# Test
vif_data = pd.DataFrame()
vif_data["Variable"] = X_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]

# Results
print("VIF for each explicative variable:")
print(vif_data)


VIF for each explicative variable:
     Variable       VIF
0       const  7.352813
1  Infan_Rate  1.001876
2  Tuber_osis  1.001876


### __5. Exogeneity__

__Las variables explicativas $X_i$ no deben estar correlacionadas con los términos de error $\epsilon_i$.__ Si las variables explicativas están correlacionadas con los errores, el modelo será endógeno y las estimaciones serán sesgadas e inconsistentes.

__Mathematical Formulation__

La exogeneidad se expresa como la no correlación entre las variables explicativas $X_i$ y los términos de error $\epsilon_i$:

$$\text{Cov}(X_i, \epsilon_i) = 0$$

Esto significa que no debe existir correlación entre las variables explicativas y los errores del modelo.

__Hypothesis__

- $H_0)$ Las variables explicativas son endógenas, es decir, están correlacionadas con los errores.

  $$ H_0: \text{Cov}(X_i, \epsilon_i) \neq 0 \quad (\text{endogeneidad presente}) $$

- $H_1)$ Las variables explicativas son exógenas, es decir, no están correlacionadas con los errores.

  $$ H_1: \text{Cov}(X_i, \epsilon_i) = 0 \quad (\text{sin endogeneidad}) $$

__Test__

Se puede usar el __Test de Hausman o el Test de Durbin-Wu-Hausman__ para verificar la exogeneidad. Estos tests comparan estimadores de Mínimos Cuadrados Ordinarios (OLS) con estimadores robustos (como los de Máxima Verosimilitud (ML)) y ayudan a identificar si las variables explicativas están correlacionadas con los errores.

Por limitaciones de este curso no veremos este supuesto a profundidad.

---